<a href="https://colab.research.google.com/github/koyomin9zx/CS331.K11-Flowers-Recognition/blob/master/FlowerRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip /content/drive/'My Drive'/ML/dataset.zip

In [0]:
!ln -s /content/drive/'My Drive'/ML/backup /content

In [0]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.externals import joblib
import cv2

In [0]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [0]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

In [0]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
model = VGG16(weights='imagenet', include_top=False)

In [0]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [0]:
# Dùng pre-trained model để lấy ra các feature của ảnh
features = model.predict(list_image)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features = features.reshape((features.shape[0], 512*7*7))

In [0]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [22]:
#train với linearSVM
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [56]:
#kiểm tra kết quả của linearSVM
y_pred = svclassifier.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.85      0.88       170
           1       0.91      0.96      0.93       216
           2       0.86      0.90      0.88       149
           3       0.86      0.84      0.85       135
           4       0.89      0.88      0.88       195

    accuracy                           0.89       865
   macro avg       0.89      0.89      0.89       865
weighted avg       0.89      0.89      0.89       865



In [27]:
#train với Polynomial Kernel
svclassifierpoly = SVC(kernel='poly', degree=8)
svclassifierpoly.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=8, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [57]:
#kiểm tra kết quả của Polynomial Kernel
y_pred_poly = svclassifierpoly.predict(X_test)
print(classification_report(y_test,y_pred_poly))

              precision    recall  f1-score   support

           0       0.84      0.46      0.60       170
           1       0.29      0.98      0.45       216
           2       0.60      0.04      0.08       149
           3       1.00      0.08      0.15       135
           4       0.85      0.12      0.21       195

    accuracy                           0.38       865
   macro avg       0.72      0.34      0.30       865
weighted avg       0.69      0.38      0.31       865



In [29]:
#Train với Gaussian Kernel
svclassifier_gaussian = SVC(kernel='rbf')
svclassifier_gaussian.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [30]:
#kiểm tra kết quả của Gaussian Kernel
y_pred_gauss = svclassifier_gaussian.predict(X_test)
print(classification_report(y_test,y_pred_gauss))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       170
           1       0.25      1.00      0.40       216
           2       0.33      0.01      0.01       149
           3       0.00      0.00      0.00       135
           4       0.00      0.00      0.00       195

    accuracy                           0.25       865
   macro avg       0.12      0.20      0.08       865
weighted avg       0.12      0.25      0.10       865



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
#Train với softmax
# Grid search để tìm các parameter tốt nhất cho model. C = 1/lamda, hệ số trong regularisation. Solver là kiểu optimize
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
model = GridSearchCV(LogisticRegression(solver='lbfgs', multi_class='multinomial'), params)
#model = GridSearchCV(LogisticRegression(), params)
model.fit(X_train, y_train)
print('Best parameter for the model {}'.format(model.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.

Best parameter for the model {'C': 0.1}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [61]:
# Đánh giá Sofmax
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.92      0.84      0.88       170
           1       0.93      0.96      0.94       216
           2       0.85      0.89      0.87       149
           3       0.87      0.86      0.86       135
           4       0.87      0.88      0.88       195

    accuracy                           0.89       865
   macro avg       0.89      0.89      0.89       865
weighted avg       0.89      0.89      0.89       865



In [62]:
#lưu các Model
joblib.dump(svclassifier_gaussian, 'backup/svclassifier_gaussian.pkl')
joblib.dump(svclassifier, 'backup/svclassifier.pkl')
joblib.dump(svclassifierpoly, 'backup/svclassifier_poly.pkl')
joblib.dump(model,'backup/softmax.pkl')

['backup/softmax.pkl']

In [168]:
#Test ảnh
def testImage(path):
  model1 = VGG16(weights='imagenet', include_top=False)
  img = cv2.imread(path)
  img = cv2.resize(img,(224,224))
  img = img_to_array(img)
  img = np.expand_dims(img, 0)
  img = imagenet_utils.preprocess_input(img)
  imgfeat = model1.predict(img)
  imgfeat = imgfeat.reshape((imgfeat.shape[0], 512*7*7))
  return imgfeat

def nameFlower(pred):
  if(pred[0]==0):
    return "Hoa Cúc"
  if(pred[0]==1):
    return "Hoa Bồ Công Anh"
  if(pred[0]==2):
    return "Hoa Hồng"
  if(pred[0]==3):
    return "Hoa Hướng Dương"
  if(pred[0]==4):
    return "Hoa Tuy Líp"



imgfeat=testImage('tulip.jpg')


preds = model.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán sofmax: ',nameFlower(preds))

preds = svclassifier.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán linearSVM: ',nameFlower(preds))

preds = svclassifier_gaussian.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán gaussian: ',nameFlower(preds))

preds = svclassifierpoly.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán poly: ',nameFlower(preds))




Giá trị dự đoán sofmax:  Hoa Tuy Líp
Giá trị dự đoán linearSVM:  Hoa Tuy Líp
Giá trị dự đoán gaussian:  Hoa Bồ Công Anh
Giá trị dự đoán poly:  Hoa Tuy Líp
